### Chapter 21
## Saving and Loading Trained Models

### 21.0 Introduction
In the last 20 chapters around 200 recipies, we have convered how to take raw data nad usem achine learning to create well-performing predictive models. However, for all our work to be worthwhile we eventually need to do something with our model, such as integrating it with an existing software application. To accomplish this goal, we need to be able to bot hsave our models after training and load them when they are needed by an application. This is the focus of the final chapter

### 21.1 Saving and Loading a scikit-learn Model
#### Problem
You have trained a scikit-learn model and want to save it and load it elsewhere.

#### Solution
Save the model as a pickle file:

In [4]:
# load libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import joblib

# load data
iris = datasets.load_iris()
features = iris.data
target = iris.target

# create decision tree classifier object
classifier = RandomForestClassifier()

# train model
model = classifier.fit(features, target)

# save model as pickle file
joblib.dump(model, "model.pkl")

ImportError: cannot import name 'joblib' from 'sklearn' (c:\users\lalbabu\appdata\local\programs\python\python37\lib\site-packages\sklearn\__init__.py)

Once the model is saved we can use scikit-learn in our destination application (e.g., web application) to load the model:

In [ ]:
# load model from file
classifier = joblib.load("model.pkl")

And use it to make predictions

In [ ]:
# create new observation
new_observation = [[ 5.2, 3.2, 1.1, 0.1]]

# predict obserrvation's class
classifier.predict(new_observation)

### Discussion
The first step in using a model in production is to save that model as a file that can be loaded by another application or workflow. We can accomplish this by saving the model as a pickle file, a Python-specific data format. Specifically, to save the model we use `joblib`, which is a library extending pickle for cases when we have large NumPy arrays--a common occurance for trained models in scikit-learn.

When saving scikit-learn models, be aware that saved models might not be compatible between versions of scikit-learn; therefore, it can be helpful to include the version of scikit-learn used in the model in the filename:

In [ ]:
# import library
import sklearn

# get scikit-learn version
scikit_version = joblib.__version__

# save model as pickle file
joblib.dump(model, "model_(version).pkl".format(version=scikit_version))

### 21.2 Saving and Loading a Keras Model
#### Problem
You have a trained Keras model and want to save it and load it elsewhere.

#### Solution
Save the model as HDF5:

In [ ]:
# load libraries
import numpy as np
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras.models import load_model

# set random seed
np.random.seed(0)

# set the number of features we want
number_of_features = 1000

# load data and target vector from movie review data
(train_Data, train_target), (test_data, test_target) = imdb.load_data(num_words=number_of_features)

# convert movie review data to a one-hot encoded feature matrix
tokenizer = Tokenizer(num_words=number_of_features)
train_features = tokenizer.sequences_to_matrix(train_data, mode="binary")
test_features = tokenizer.sequences_to_matrix(test_data, mode="binary")

# start neural network
network = models.Sequential()

# add fully connected layer with ReLU activation function
network.add(layers.Dense(units=16, activation="relu", input_shape=(number_of_features,)))

# add fully connected layer with a sigmoid activation function
network.add(layers.Dense(units=1, activation="sigmoid"))

# compile neural network
network.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

# train neural network
history = network.fit(train_features, train_target, epochs=3, verbose=0, batch_size=100, validation_data=(test_features, test_target))

# save neural network
network.save("model.h5")

We can then load the model either in another application or for additional training

In [ ]:
# load neural network
network = load_model("model.h5")

#### Discussion
Unlike scikit-learn, Keras does not recommend you save models using pickle. Instead, models are saved as an HDF5 file. The HDF5 file contains everything you need to not only load the model to make predicitons (i.e., achitecture and trained parameters), but also to restart training (i.e. loss and optimizer settings and the current state)